In [0]:
!pip install langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/730.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 23.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

Agent Use Cases:
1. Lodging - Airbnb
2. Lodging - Booking
3. Lodging - Google
4. Itinerary
5. Location to Location

Need to determine lodging first or itinerary first
Location to location will always come last

In [0]:
from openai import OpenAI
import os
from IPython.display import display, Markdown as md

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
# DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
# Alternatively in a Databricks notebook you can use this:
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://dbc-853dca7e-4798.cloud.databricks.com/serving-endpoints"
)

response = client.chat.completions.create(
    model="Maverick-Hackathon",
    messages=[
        {
            "role": "user",
            "content": "What is an LLM agent?"
        }
    ]
)

print(response.choices[0].message.content)

An LLM agent is a software entity that utilizes a Large Language Model (LLM) to perform various tasks. To understand what an LLM agent is, let's break down the components:

1. **Large Language Model (LLM)**: LLMs are a type of artificial intelligence (AI) model designed to process and generate human-like language. These models are trained on vast amounts of text data, enabling them to understand and respond to a wide range of inputs, from simple queries to complex tasks.

2. **Agent**: In the context of computing and AI, an agent refers to a program or system that can perceive its environment, make decisions based on its perceptions, and act upon its environment to achieve certain goals. Agents can be simple or complex, ranging from a basic software bot that performs a routine task to sophisticated systems that can learn and adapt.

When we talk about an **LLM agent**, we're referring to a software agent that leverages the capabilities of a Large Language Model. This means the agent ca

In [0]:
from langchain.agents import initialize_agent, AgentType
from langchain.chains import SequentialChain

In [0]:
def call_llm(system_prompt, inputs):
    data = {
        "model": "Maverick-Hackathon",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": str(inputs)
            }
        ]
    }
    response = client.chat.completions.create(**data)
    return response.choices[0].message.content

In [0]:
def generate_itinerary(accessibility_needs, destination, dates):
    system_prompt = (
        f"You are a helpful travel agent. Your task is to generate a travel itinerary for a user with specific accessibility needs. "
        f"The user may not have specific plans in mind. The user is arriving to {destination}. "
        f"Date Context {dates if len(dates) > 0 else 'No dates provided'}. Accessibility Needs: {accessibility_needs}."
    )
    inputs = {
        "accessibility_needs": accessibility_needs,
        "destination": destination,
        "dates": dates
    }
    return call_llm(system_prompt, inputs)

def find_lodging(accessibility_needs, destination, dates):
    system_prompt = (
        "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. "
        "Ensure that the lodging has the accomodations necessary for the user. If you cannot meet those requirements, then return 'No lodging with accomodations found'."
    )
    inputs = {
        "accessibility_needs": accessibility_needs,
        "destination": destination,
        "dates": dates
    }
    return call_llm(system_prompt, inputs)

def create_booking(lodging_info, accessibility_needs, user_info):
    system_prompt = (
        f"Given the lodging information and trip context. Pre-fill the form for the user. "
        f"Context: {lodging_info}; User Information: {user_info}."
    )
    inputs = {
        "lodging_info": lodging_info,
        "accessibility_needs": accessibility_needs,
        "user_info": user_info
    }
    return call_llm(system_prompt, inputs)

def generate_directions(origin, destination, accessibility_needs, date):
    system_prompt = (
        f"Given that the user is going to {destination} on {date}, generate detailed, accessible directions from their home to the destination. "
        f"Allow ample time and include step-by-step accessible options for the following needs: {accessibility_needs}."
    )
    inputs = {
        "origin": origin,
        "destination": destination,
        "accessibility_needs": accessibility_needs,
        "date": date
    }
    return call_llm(system_prompt, inputs)

def generate_overall_plan(accessibility_needs, destination, dates, user_info, itinerary):
    system_prompt = (
        f"Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. "
        f"Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each day as a separate response, so that way you can maximize the information provided to the user. "
        f"Provide the location or lodging website URL. Lodging: {{lodging}}, Booking: {{booking}}, Directions: {{directions}}."
    )
    inputs = {
        "accessibility_needs": accessibility_needs,
        "destination": destination,
        "dates": dates,
        "user_info": user_info
    }
    return call_llm(system_prompt, inputs)

In [0]:
class Step:
    def __init__(self, name, function, inputs, system_prompt):
        self.name = name
        self.function = function
        self.inputs = inputs
        self.system_prompt = system_prompt
        self.output = None
        self.input_keys = tuple(inputs)

    def run(self):
        # Implement the logic to execute the function with inputs
        self.output = self.function(*self.inputs)
        return self.output
    
class TravelAgent:
    def __init__(self, accessibility_needs, destination, dates, user_info):
        self.accessibility_needs = accessibility_needs
        self.destination = destination
        self.dates = dates
        self.user_info = user_info
        self.lodging_info = None
        self.high_level_itineraries = []

    def generate_itinerary_step(self, date):
        return Step(
            name="Generate Itinerary",
            function=generate_itinerary,
            inputs=[self.accessibility_needs, self.destination, [date]],
            system_prompt = "You are a helpful travel agent. Your task is to generate a high-level travel itinerary for a user with specific accessibility needs for the date {date}. The user may not have specific plans in mind, so prioritize convenience and accessibility while viewing the key attractions of each destination. Include 1-2 activities and nearby meal options. The user is arriving to {self.destination}. Date Context {date}. Accessibility Needs: {self.accessibility_needs}."
        )

    def find_lodging_step(self):
        return Step(
            name="Find Lodging",
            function=find_lodging,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. Ensure that the lodging has the accommodations necessary for the user, if you cannot meet those requirements, then return 'No lodging with accommodations found'."
        )

    def create_booking_step(self):
        return Step(
            name="Create Booking",
            function=create_booking,
            inputs=[self.lodging_info, self.accessibility_needs, self.user_info],
            system_prompt = "Given the lodging information and trip context. Pre-fill the form for the user. Context: {self.lodging_info}; User Information: {self.user_info}."
        )

    def generate_directions_step(self, start_location, end_location, date):
        return Step(
            name="Generate Directions",
            function=generate_directions,
            inputs=[start_location, end_location, self.accessibility_needs, date],
            system_prompt = "Given that the user is going to {end_location} on {date}, generate detailed, accessible directions from {start_location} to the destination. Allow ample time and include step-by-step accessible options for the following needs: {self.accessibility_needs}."
        )

    def generate_overall_plan_step(self, date, itinerary):
        return Step(
            name="Generate Overall Plan",
            function=generate_overall_plan,
            inputs=[self.accessibility_needs, self.destination, [date], self.user_info, itinerary],
            system_prompt = "Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each date as a separate response, so that way you can maximize the information provided to the user. Provide the location or lodging website URL. Lodging: {self.find_lodging_step().output}, Booking: {self.create_booking_step().output}."
        )

    def create_chain(self):
        lodging_step = self.find_lodging_step()
        self.lodging_info = lodging_step.run()  # Update lodging_info attribute
        
        booking_step = self.create_booking_step()
        booking_output = booking_step.run()
        
        # Generate high-level itineraries for each day
        for date in self.dates:
            itinerary_step = self.generate_itinerary_step(date)
            high_level_itinerary = itinerary_step.run()
            self.high_level_itineraries.append((date, high_level_itinerary))
        
        overall_plans = []
        for date, high_level_itinerary in self.high_level_itineraries:
            overall_plan_step = self.generate_overall_plan_step(date, high_level_itinerary)
            day_plan = overall_plan_step.run()
            overall_plans.append(day_plan)
        
        return overall_plans

In [0]:
# Example usage
accessibility_needs = "wheelchair access, visual aids"
destination = "Paris"
dates = ["2025-06-10", "2025-06-11", "2025-06-12"]
user_info = {"name": "John Doe", "contact": "john.doe@example.com"}

travel_agent = TravelAgent(accessibility_needs, destination, dates, user_info)
travel_chain = travel_agent.create_chain()
md('\n'.join(item for item in travel_chain))

_1
"Based on your accessibility needs, I've created a comprehensive travel plan for your trip to Paris on June 10, 2025. I'll provide you with a detailed itinerary, day by day. **Day 1: June 10, 2025** ### Lodging You will be staying at the **Novotel Paris Tour Eiffel** hotel, which is wheelchair accessible and has adapted rooms. The hotel is equipped with: - Ramps and elevators for easy navigation - Wide doors and corridors - Adapted bathrooms with grab bars and lowered sinks Lodging: [Novotel Paris Tour Eiffel](https://www.novotel.com/en/hotel-6816-novotel-paris-tour-eiffel/index.shtml) Booking: You can book your room through the Novotel website or through a booking platform like Booking.com. Ensure to request an adapted room. Directions: The hotel is located at 61 Quai de Grenelle, 75015 Paris. You can take the Metro Line 6 to Bir-Hakeim station, and it's a short walk from there. ### Itinerary 1. **Morning: Arrival and Check-in** - Arrive at the Novotel Paris Tour Eiffel. - Check-in and get familiar with the hotel's facilities. - The hotel staff can provide visual aids or assistance if needed. 2. **Afternoon: Eiffel Tower Visit** - The Eiffel Tower is wheelchair accessible with ramps and elevators. - You can book tickets in advance through the official Eiffel Tower website: [Eiffel Tower Tickets](https://www.toureiffel.paris/en/accessibility). - The Eiffel Tower is about a 10-minute walk from the hotel. 3. **Evening: Dinner at a Nearby Accessible Restaurant** - **Le Jules Verne** is a Michelin-starred restaurant located on the second floor of the Eiffel Tower, with wheelchair access and stunning views. - Booking: [Le Jules Verne](https://www.restaurantlejulesverne.com/en/reservation) - Alternatively, you can explore other accessible dining options near the hotel. ### Additional Tips - Ensure to carry your accessibility needs documentation or a doctor's note, if applicable, to facilitate any assistance required during your trip. - The Paris Metro has some accessible stations, but not all. Plan your route in advance using an accessibility-focused journey planner. For any further assistance or to make adjustments to this travel plan, please do not hesitate to contact me."
"**Paris Travel Plan for John Doe (2025-06-11)** ### Day 1: 2025-06-11 #### Lodging You will be staying at the **Hotel Le Walt**, which is known for its wheelchair accessibility. The hotel has adapted rooms with roll-in showers and lowered sinks. - **Lodging:** Hotel Le Walt - **Booking:** You can book your stay through their official website: [Hotel Le Walt Booking](https://www.hotellewalt.com/en/reservation/) - **Directions:** Located at 8 Rue De L'Université, 75007 Paris. You can find directions on their website: [Hotel Le Walt Directions](https://www.hotellewalt.com/en/location/) #### Itinerary 1. **Breakfast at Le Walt** - Start your day with breakfast at the hotel. They offer a buffet that is accessible for guests with mobility impairments. Visual aids such as menus in braille are available upon request. 2. **The Louvre Museum** - After breakfast, head to the Louvre Museum. The museum is wheelchair accessible with ramps and elevators. Wheelchairs are also available for loan on a first-come, first-served basis. - **Directions:** The Louvre is approximately a 15-minute walk from Hotel Le Walt. You can navigate using accessible pedestrian paths. For exact directions, you can use a GPS or visit the Louvre's website: [Louvre Museum Directions](https://www.louvre.fr/en/plan-louvre-museum) - **Accessibility:** The Louvre provides various visual aids, including audio descriptions and tactile models for visually impaired visitors. 3. **Lunch at Le Comptoir du Relais** - For lunch, visit Le Comptoir du Relais, a restaurant that has wheelchair access. Although it's a bit narrow, the staff is accommodating to guests with disabilities. - **Booking:** It's advisable to book in advance to ensure they can accommodate your accessibility needs: [Le Comptoir du Relais Boo

Based on your accessibility needs, I've created a comprehensive travel plan for your trip to Paris on June 10, 2025. I'll provide you with a detailed itinerary, day by day.

**Day 1: June 10, 2025**

### Lodging
You will be staying at the **Novotel Paris Tour Eiffel** hotel, which is wheelchair accessible and has adapted rooms. The hotel is equipped with:
- Ramps and elevators for easy navigation
- Wide doors and corridors
- Adapted bathrooms with grab bars and lowered sinks

Lodging: [Novotel Paris Tour Eiffel](https://www.novotel.com/en/hotel-6816-novotel-paris-tour-eiffel/index.shtml)
Booking: You can book your room through the Novotel website or through a booking platform like Booking.com. Ensure to request an adapted room.
Directions: The hotel is located at 61 Quai de Grenelle, 75015 Paris. You can take the Metro Line 6 to Bir-Hakeim station, and it's a short walk from there.

### Itinerary
1. **Morning: Arrival and Check-in**
   - Arrive at the Novotel Paris Tour Eiffel.
   - Check-in and get familiar with the hotel's facilities.
   - The hotel staff can provide visual aids or assistance if needed.

2. **Afternoon: Eiffel Tower Visit**
   - The Eiffel Tower is wheelchair accessible with ramps and elevators.
   - You can book tickets in advance through the official Eiffel Tower website: [Eiffel Tower Tickets](https://www.toureiffel.paris/en/accessibility).
   - The Eiffel Tower is about a 10-minute walk from the hotel.

3. **Evening: Dinner at a Nearby Accessible Restaurant**
   - **Le Jules Verne** is a Michelin-starred restaurant located on the second floor of the Eiffel Tower, with wheelchair access and stunning views.
   - Booking: [Le Jules Verne](https://www.restaurantlejulesverne.com/en/reservation)
   - Alternatively, you can explore other accessible dining options near the hotel.

### Additional Tips
- Ensure to carry your accessibility needs documentation or a doctor's note, if applicable, to facilitate any assistance required during your trip.
- The Paris Metro has some accessible stations, but not all. Plan your route in advance using an accessibility-focused journey planner.

For any further assistance or to make adjustments to this travel plan, please do not hesitate to contact me.
**Paris Travel Plan for John Doe (2025-06-11)**

### Day 1: 2025-06-11

#### Lodging
You will be staying at the **Hotel Le Walt**, which is known for its wheelchair accessibility. The hotel has adapted rooms with roll-in showers and lowered sinks. 
- **Lodging:** Hotel Le Walt
- **Booking:** You can book your stay through their official website: [Hotel Le Walt Booking](https://www.hotellewalt.com/en/reservation/)
- **Directions:** Located at 8 Rue De L'Université, 75007 Paris. You can find directions on their website: [Hotel Le Walt Directions](https://www.hotellewalt.com/en/location/)

#### Itinerary

1. **Breakfast at Le Walt**
   - Start your day with breakfast at the hotel. They offer a buffet that is accessible for guests with mobility impairments. Visual aids such as menus in braille are available upon request.

2. **The Louvre Museum**
   - After breakfast, head to the Louvre Museum. The museum is wheelchair accessible with ramps and elevators. Wheelchairs are also available for loan on a first-come, first-served basis.
   - **Directions:** The Louvre is approximately a 15-minute walk from Hotel Le Walt. You can navigate using accessible pedestrian paths. For exact directions, you can use a GPS or visit the Louvre's website: [Louvre Museum Directions](https://www.louvre.fr/en/plan-louvre-museum)
   - **Accessibility:** The Louvre provides various visual aids, including audio descriptions and tactile models for visually impaired visitors.

3. **Lunch at Le Comptoir du Relais**
   - For lunch, visit Le Comptoir du Relais, a restaurant that has wheelchair access. Although it's a bit narrow, the staff is accommodating to guests with disabilities.
   - **Booking:** It's advisable to book in advance to ensure they can accommodate your accessibility needs: [Le Comptoir du Relais Booking](https://www.comptoir-relais.com/en/reservation)

4. **Palais-Royal**
   - After lunch, visit the Palais-Royal, which is fully wheelchair accessible. The complex includes beautiful gardens and contemporary art spaces.
   - **Directions:** Palais-Royal is about a 10-minute walk from Le Comptoir du Relais. Use accessible pedestrian paths and follow GPS directions: [Palais-Royal Directions](https://www.domaine-palais-royal.fr/en/plan-your-visit/accessibility)

5. **Dinner at Le Grand Vefour**
   - For dinner, dine at Le Grand Vefour, a historic restaurant that is wheelchair accessible. It has a lift for guests with mobility impairments.
   - **Booking:** Make sure to book in advance to ensure accessibility arrangements: [Le Grand Vefour Booking](https://www.grand-vefour.com/en/reservation)

#### Accessibility Tips
- Ensure you have your visual aids and any personal assistance devices with you.
- Inform your lodging and restaurants/destinations about your accessibility needs in advance.

#### Contact Information
For any last-minute changes or assistance during your trip, you can contact John Doe at john.doe@example.com.

Enjoy your accessible trip to Paris!
**Paris Travel Plan for John Doe (2025-06-12)**

### Day 1: 2025-06-12

#### Morning: Arrival and Check-in

*   **Lodging:** Hotel Le Walt **** 
    *   **Booking:** [Book on Booking.com](https://www.booking.com/hotel/fr/hotel-le-walt.html?aid=304142;label=gen173nr-1DCAEogIAKAAmBmIAg2AADOYAQmyaHX6AQIYAQGYAQm4AQQgAQH4AQKIAgGoAgO4Aq3Rl5oGwAIB0gIKAFgAcABLw7Y;sid=8f49569f9d70e0c3f0a9ed0b9b0a455b;dest_id=-1456928;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;hpos=1;no_rooms=1;room1=A%2CA;sb_price_type=total;sr_order=price;sr_pri_blocks=295%2C168%2Cfalse%2C295%2C168;srepoch=1710279558;srpvid=d3d4e8d3e351006b;type=total;ucfs=1&#hotelTmpl)
    *   **Website:** [Hotel Le Walt Official Website](https://www.hotel-lewalt.com/)
    *   **Directions:** Hotel Le Walt is located at 8 Rue De L'Amiral Hamelin, 75016 Paris, France. It is easily accessible by public transportation or taxi. The nearest metro station is Iéna (ligne 9), about a 5-minute walk away.
*   **Accessibility Features:** 
    *   Wheelchair-accessible rooms and public areas
    *   Elevator with voice announcements and Braille buttons
    *   Visual aids available upon request

#### Afternoon: Explore the Eiffel Tower

*   **Eiffel Tower (Tour Eiffel)**
    *   **Address:** 5 Avenue Anatole France, 75007 Paris, France
    *   **Accessibility:** 
        *   Ramps and elevators for wheelchair access
        *   Adapted restrooms
        *   Audio descriptions and sign language interpretation available for guided tours (book in advance)
    *   **Directions:** From Hotel Le Walt, take the metro (ligne 9) from Iéna to Trocadéro, then follow the signs to the Eiffel Tower. Approximately a 10-minute journey.

#### Evening: Seine River Cruise

*   **Bateaux-Mouches**
    *   **Departure Point:** Pont de l'Alma, 75008 Paris, France
    *   **Accessibility:** 
        *   Wheelchair-accessible boarding ramps
        *   Adapted restrooms on board
        *   Audio descriptions available in multiple languages
    *   **Directions:** From the Eiffel Tower, walk towards Pont de l'Alma. The departure point is clearly marked. Approximately a 15-minute walk.

For any changes or special requests, please contact John Doe at john.doe@example.com. We look forward to assisting you further.